# Test Best Models

Test notebook to generate the statistics of the different models found with AMS. First load the required libraries

In [1]:
import datetime
import logging
import sys
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from keras.callbacks import LearningRateScheduler
from keras.optimizers import Adam
import keras.backend as K
from keras.models import model_from_json

from sklearn.model_selection import KFold
from scipy import stats

#sys.path.append('/Users/davidlaredorazo/Documents/University_of_California/Research/Projects')
sys.path.append('/media/controlslab/DATA/Projects')

import ann_framework.aux_functions as aux_functions

import automatic_model_selection
from automatic_model_selection import Configuration
from ann_encoding_rules import Layers
import fetch_to_keras
#from CMAPSAuxFunctions import TrainValTensorBoard

#Tunable model
from ann_framework.tunable_model.tunable_model import SequenceTunableModelRegression, SequenceTunableModelClassification

#Data handlers
from ann_framework.data_handlers.data_handler_CMAPSS import CMAPSSDataHandler
from ann_framework.data_handlers.data_handler_MNIST import MNISTDataHandler
from ann_framework.data_handlers.data_handler_CIFAR10 import CIFAR10DataHandler

Using TensorFlow backend.


### Given a model, get the compiled model

In [2]:
def get_compiled_model(model, problem_type, optimizer_params=[]):
    """Obtain a keras compiled model"""
    
    #Shared parameters for the models
    optimizer = Adam(lr=0.001, beta_1=0.5)
    
    if problem_type == 1:
        lossFunction = "mean_squared_error"
        metrics = ["mse"]
    elif problem_type == 2:
        lossFunction = "categorical_crossentropy"
        metrics = ["accuracy"]
    else:
        print("Problem type not defined")
        model = None
        return
    
    #Create and compile the models
    model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    
    return model


def create_tunable_model(model_genotype, problem_type, input_shape, data_handler, model_number):
    
    K.clear_session()
    
    model = fetch_to_keras.decode_genotype(model_genotype, problem_type, input_shape, 1)
    
    model = get_compiled_model(model, problem_type, optimizer_params=[])
    
    if problem_type == 1:
        tModel = SequenceTunableModelRegression('ModelReg_SN_'+str(model_number), model, lib_type='keras', data_handler=data_handler)
    else:
        tModel = SequenceTunableModelClassification('ModelClass_SN_'+str(model_number), model, lib_type='keras', data_handler=data_handler)
        
    return tModel

### Load cmaps data handler

In [3]:
def cmaps_dhandler(data_scaler=None):

    #Selected as per CNN paper
    features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 'BPR', 
    'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
    selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
    selected_features = list(features[i] for i in selected_indices-1)
    data_folder = '../CMAPSSData'

    window_size = 25
    window_stride = 1
    max_rul = 130

    dHandler_cmaps = CMAPSSDataHandler(data_folder, 1, selected_features,
                                       max_rul, window_size, window_stride, data_scaler=data_scaler)

    input_shape = (len(selected_features)*window_size, )

    return dHandler_cmaps

### Load models and evaluate model

In [4]:
def load_model(model_file, weights_file="", problem_type=1):
    
    p_type = ""
    
    # load json and create model
    json_file = open(model_file, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    print("Loaded model from disk") 
        
    #Load weights if specified
    if weights_file != "":
        # load weights into new model
        loaded_model.load_weights(weights_file)
        print("Loaded weights from disk") 
    else:
        print("Model needs training")
        
    optimizer = Adam(lr=0.001, beta_1=0.5)
    
    if problem_type == 1:
        p_type = "regression"
        lossFunction = "mean_squared_error"
        metrics = ["mse"]
    elif problem_type == 2:
        p_type = "classification"
        lossFunction = "categorical_crossentropy"
        metrics = ["accuracy"]
    else:
        print("Problem type not defined")
        model = None
        return
    
    #Create and compile the models
    loaded_model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    print("Created model for " + p_type + " with loss function " + lossFunction)

    return loaded_model
        
        

### Load each of the models and test them

In [5]:
best_model_folder = 'best_models'
test_sets = {'cifar10':(CIFAR10DataHandler, None, 2), 
             'cmapss':(cmaps_dhandler, MinMaxScaler(feature_range=(-1, 1)), 1), 
             'mnist':((MNISTDataHandler), None, 2)}
alpha_folders = ['alpha0.6', 'alpha0.8', 'alpha1']

In [6]:
results = {}

k = 2  #For 10-fold cross validation

model_file_name = 'bestModel_global.json'
weights_file_name = 'bestModel_global.h5'

for dataset in test_sets.keys():
    for alphaValue in alpha_folders:
        
        results_key = dataset + "_" +  alphaValue
        
        evaluations_cv = np.zeros(k)
        evaluations_test = np.zeros(k)
        
        model_location = best_model_folder + '/' + dataset + '/' + alphaValue + '/' + model_file_name
        
        if weights_file_name != "":
            weights_location = best_model_folder + '/' + dataset + '/' + alphaValue + '/' + weights_file_name
        
        dhandler, data_scaler, problem_type = test_sets[dataset]

        #model = load_model(model_location, weights_location, problem_type)   
        
        data_handler = dhandler(data_scaler=data_scaler)
        data_handler.load_data(verbose = 1, )
        data_handler.print_data()
        
        folds = list(KFold(n_splits=k, shuffle=True).split(data_handler.X_train))
        
        print('Validation on model:' + model_location)
        
        for j, (train_idx, val_idx) in enumerate(folds):

            print('\nExperiment on Fold ', j)
            
            K.clear_session()  #Clear the previous tensorflow graph 

            X_train_cv = data_handler.X_train[train_idx]
            y_train_cv = data_handler.y_train[train_idx]
            X_valid_cv = data_handler.X_train[val_idx]
            y_valid_cv = data_handler.y_train[val_idx]

            model = load_model(model_location, "", problem_type)
            model.summary()

            model.fit(X_train_cv, y_train_cv, batch_size=512, epochs=100, verbose=1)

            evaluation_cv = model.evaluate(X_valid_cv, y_valid_cv)
            evaluation_test = model.evaluate(data_handler.X_test, data_handler.y_test)

            evaluations_cv[j] = evaluation_cv[1]
            evaluations_test[j] = evaluation_test[1]
            
        results[results_key] = (evaluations_cv, evaluations_test)
        

Loading data. Cros-Validation ratio 0
Printing shapes

Training data (X, y)
(50000, 3072)
(50000, 10)
Testing data (X, y)
(10000, 3072)
(10000, 10)
Printing first 5 elements

Training data (X, y)
[[0.23137255 0.24313726 0.24705882 ... 0.48235294 0.36078432 0.28235295]
 [0.6039216  0.69411767 0.73333335 ... 0.56078434 0.52156866 0.5647059 ]
 [1.         1.         1.         ... 0.3137255  0.3372549  0.32941177]
 [0.10980392 0.09803922 0.03921569 ... 0.28235295 0.25490198 0.18039216]
 [0.6666667  0.7058824  0.7764706  ... 0.28627452 0.3019608  0.3137255 ]]
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
Testing data (X, y)
[[0.61960787 0.4392157  0.19215687 ... 0.08235294 0.2627451  0.43137255]
 [0.92156863 0.92156863 0.92156863 ... 0.7294118  0.78431374 0.78039217]
 [0.61960787 0.74509805 0.87058824 ... 0.02745098 0.03137255 0.02745098]
 [0.60784316 0.6117647  0.5843137

25000/25000 [==============================] - 0s 7us/step - loss: 1.2409 - acc: 0.5748
Epoch 62/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.2366 - acc: 0.5744
Epoch 63/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.2325 - acc: 0.5764
Epoch 64/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.2255 - acc: 0.5781
Epoch 65/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.2170 - acc: 0.5787
Epoch 66/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.2214 - acc: 0.5777
Epoch 67/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.2167 - acc: 0.5789
Epoch 68/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.1983 - acc: 0.5885
Epoch 69/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.1971 - acc: 0.5897
Epoch 70/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.2004 - acc: 0.5862
Epo

25000/25000 [==============================] - 0s 7us/step - loss: 1.4463 - acc: 0.4966
Epoch 34/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.4368 - acc: 0.4980
Epoch 35/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.4298 - acc: 0.5024
Epoch 36/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.4274 - acc: 0.5034
Epoch 37/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.4234 - acc: 0.5021
Epoch 38/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.4182 - acc: 0.5076
Epoch 39/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.4086 - acc: 0.5109
Epoch 40/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.3949 - acc: 0.5150
Epoch 41/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.3907 - acc: 0.5151
Epoch 42/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.3886 - acc: 0.5164
Epo

Loaded model from disk
Model needs training
Created model for classification with loss function categorical_crossentropy
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 56)                172088    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                570       
Total params: 172,658
Trainable params: 172,658
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
25000/25000 [==============================] - 0s 11us/step - loss: 2.1092 - acc: 0.2368
Epoch 2/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.9345 - acc: 0.3241
Epoch 3/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.8678 - acc: 0.3496
Epoch 4/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.8298 - acc: 0.3634
Epoch 5/10

25000/25000 [==============================] - 0s 7us/step - loss: 1.2914 - acc: 0.5518
Epoch 76/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.2934 - acc: 0.5499
Epoch 77/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.2887 - acc: 0.5526
Epoch 78/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.2807 - acc: 0.5550
Epoch 79/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.2772 - acc: 0.5590
Epoch 80/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.2719 - acc: 0.5578
Epoch 81/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.2729 - acc: 0.5575
Epoch 82/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.2692 - acc: 0.5579
Epoch 83/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.2718 - acc: 0.5570
Epoch 84/100
25000/25000 [==============================] - 0s 7us/step - loss: 1.2696 - acc: 0.5590
Epo

...

30000/30000 [==============================] - 0s 4us/step - loss: 0.0062 - acc: 0.9998
Epoch 52/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0057 - acc: 0.9999
Epoch 53/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0053 - acc: 0.9999
Epoch 54/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0049 - acc: 0.9999
Epoch 55/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0046 - acc: 1.0000
Epoch 56/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0043 - acc: 0.9999
Epoch 57/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0041 - acc: 1.0000
Epoch 58/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0038 - acc: 1.0000
Epoch 59/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0036 - acc: 1.0000
Epoch 60/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0033 - acc: 1.0000
Epo

30000/30000 [==============================] - 0s 4us/step - loss: 0.0627 - acc: 0.9838
Epoch 22/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0583 - acc: 0.9855
Epoch 23/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0547 - acc: 0.9863
Epoch 24/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0507 - acc: 0.9877
Epoch 25/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0473 - acc: 0.9888
Epoch 26/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0439 - acc: 0.9901
Epoch 27/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0407 - acc: 0.9915
Epoch 28/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0382 - acc: 0.9917
Epoch 29/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0353 - acc: 0.9929
Epoch 30/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0325 - acc: 0.9935
Epo

10000/10000 [==============================] - 0s 14us/step
Loading data. Cross-Validation ratio 0
Printing shapes

Training data (X, y)
(60000, 784)
(60000, 10)
Testing data (X, y)
(10000, 784)
(10000, 10)
Printing first 5 elements

Training data (X, y)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Testing data (X, y)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
Validation on model:best_models/mnist/alpha1/bestModel_global.json

Experiment on Fold  0
Loaded model from disk
Model needs training
Created

30000/30000 [==============================] - 0s 4us/step - loss: 0.0208 - acc: 0.9955
Epoch 65/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0203 - acc: 0.9960
Epoch 66/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0191 - acc: 0.9963
Epoch 67/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0186 - acc: 0.9965
Epoch 68/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0186 - acc: 0.9967
Epoch 69/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0176 - acc: 0.9966
Epoch 70/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0163 - acc: 0.9972
Epoch 71/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0156 - acc: 0.9973
Epoch 72/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0156 - acc: 0.9971
Epoch 73/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0145 - acc: 0.9978
Epo

30000/30000 [==============================] - 0s 4us/step - loss: 0.0696 - acc: 0.9804
Epoch 36/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0683 - acc: 0.9815
Epoch 37/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0654 - acc: 0.9825
Epoch 38/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0640 - acc: 0.9824
Epoch 39/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0618 - acc: 0.9830
Epoch 40/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0599 - acc: 0.9843
Epoch 41/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0569 - acc: 0.9846
Epoch 42/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0561 - acc: 0.9853
Epoch 43/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0544 - acc: 0.9848
Epoch 44/100
30000/30000 [==============================] - 0s 4us/step - loss: 0.0528 - acc: 0.9866
Epo

### Print the stats

In [7]:
for key in results.keys():
    
    print("\nStats for: ")
    print(key)
    evaluations_cv, evaluations_test = results[key]
    
    print("\nCrossVal stats: ")
    print(evaluations_cv)
    print(stats.describe(evaluations_cv))
    
    print("\nTest stats: ")
    print(stats.describe(evaluations_test))


Stats for: 
cifar10_alpha0.6

CrossVal stats: 
[0.48228 0.47912]
DescribeResult(nobs=2, minmax=(0.47912, 0.48228), mean=0.4807, variance=4.992799999999988e-06, skewness=-5.263841545770484e-14, kurtosis=-2.0)

Test stats: 
DescribeResult(nobs=2, minmax=(0.4722, 0.4731), mean=0.47265, variance=4.0500000000001073e-07, skewness=0.0, kurtosis=-2.0)

Stats for: 
cifar10_alpha0.8

CrossVal stats: 
[0.45608 0.45384]
DescribeResult(nobs=2, minmax=(0.45384, 0.45608), mean=0.45496000000000003, variance=2.50879999999992e-06, skewness=-7.433231620042076e-14, kurtosis=-2.0)

Test stats: 
DescribeResult(nobs=2, minmax=(0.4573, 0.462), mean=0.45965, variance=1.1045000000000176e-05, skewness=0.0, kurtosis=-2.0)

Stats for: 
cifar10_alpha1

CrossVal stats: 
[0.4702  0.45056]
DescribeResult(nobs=2, minmax=(0.45056, 0.4702), mean=0.46038, variance=0.00019286479999999981, skewness=0.0, kurtosis=-2.0)

Test stats: 
DescribeResult(nobs=2, minmax=(0.4517, 0.4788), mean=0.46525, variance=0.0003672050000000003